**Este script genera un dataset en el que los comentarios descargados tendrán una evaluación de su temática y su emoción**.

Se ha ejecutado en Kaggle, por lo que dado que es un servicio que tiene limites en el uso de GPU semanal y por sesión, se ha ejecutado en múltiples tandas hasta procesar todos los comentarios descargados.

In [ ]:
import pandas as pd
combined_df = pd.read_csv("/kaggle/input/scientific-pendientes/pendientes.csv")
combined_df

**Evaluar lenguaje de cada comentario**

In [2]:
import torch
from transformers import pipeline

2025-05-06 18:15:03.260631: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746555303.506075      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746555303.575674      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
import torch
from transformers import pipeline

# Verifica si hay una GPU disponible
device = "cuda" if torch.cuda.is_available() else "cpu"

# Crear el pipeline en GPU si está disponible
language_detector = pipeline(
    "text-classification",
    model="papluca/xlm-roberta-base-language-detection",
    truncation=True  # Activar truncamiento
)


# Función para clasificar idiomas en "spanish", "english" o "other language"
def classify_language(text):
    if pd.isna(text) or text.strip() == "":
        return "unclassifiable"  # Manejo de valores vacíos
    result = language_detector(text)[0]  # Obtener la clasificación del modelo
    label = result["label"]

    # Asignar categorías según el resultado
    if label == "en":
        return "en"
    if label == "es":
        return "es"
    else:
        return "other language"

# Aplicar la función a la columna "comment"
combined_df["language"] = combined_df["comment"].apply(classify_language)

In [ ]:
combined_df["language"].value_counts()

**Evaluar el sentimiento de cada comentario**language_detector = pipeline(
    "text-classification",
    model="papluca/xlm-roberta-base-language-detection",
    device=0 if torch.cuda.is_available() else -1,
    truncation=True  # Activar truncamiento
)

In [ ]:
from transformers import pipeline
import torch

device = 0 if torch.cuda.is_available() else -1  # Usa la GPU si está disponible

classifier = pipeline('sentiment-analysis', 
                      model="nlptown/bert-base-multilingual-uncased-sentiment",
                      device=device)


In [ ]:
combined_df['comment'] = combined_df['comment'].astype(str)
comments = combined_df['comment'].tolist()
results = classifier(comments, truncation=True, batch_size=8)
results = pd.DataFrame(results)
results['label'] = results['label'].map({
    "1 star": "Negativo",
    "2 stars": "Negativo",
    "3 stars": "Neutral",
    "4 stars": "Positivo",
    "5 stars": "Positivo"
})

    
combined_df['emotion'] = results['label']


In [ ]:
print(combined_df['emotion'].value_counts())

In [3]:
from transformers import pipeline
import torch

# Configurar el uso de GPU si está disponible
device = 0 if torch.cuda.is_available() else -1

# Definir etiquetas
#etiquetas = ["Tratamiento médico", "Tratamiento pseudocientífico", "Apoyo emocional", "Solicitud de ayuda", "Narrativa Personal", "Consejo", "Crítica",  "Religión", "Política", "Agradecimiento", "Felicitación"]

etiquetas = ["Comparación",
             "Intercambio de información",
             "Petición de consejo",
             "Dando un consejo",
             "Primera impresión",
             "Petición de opinión",
             "Opinión",
             "Insulto",
             "Cumplido",
             "Crítica",
             "Tributo",
             "Especulación",
             "Expresión de sentimientos",
             "Enhorabuena",
             "Agradecimiento",
             "Broma",
             "Anécdota",
             "Planes",
             "Deseos",
             "Anticipación de un evento",
             "Inclasificable",
             "Spam",
             "Tratamiento médico",
             "Tratamiento pseudocientífico",
             "Apoyo emocional"]


# Inicializar el pipeline con GPU
clasificador = pipeline(
    "zero-shot-classification",
    model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli",
    device=device
)

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# Procesar comentarios en batch en lugar de uno por uno
#comentarios = combined_df['comment'].tolist()  # Convertir la columna a lista
#resultados = clasificador(comentarios, candidate_labels=etiquetas)  # Enviar batch a GPU

In [4]:
comentarios = combined_df['comment'].tolist()
total_comentarios = len(comentarios)
batch_size = 8  # Ajusta según tu GPU
resultados = []

# Frecuencia de logs (cada 5 batches, ajustable)
log_frequency = 5

print(f"🚀 Iniciando procesamiento de {total_comentarios} comentarios (batch_size={batch_size})...\n")

# Porcentaje de carga con prints básicos
for i in range(0, total_comentarios, batch_size):
    batch = comentarios[i:i + batch_size]
    resultados.extend(clasificador(batch, candidate_labels=etiquetas))
    
    # Lógica de progreso
    procesados = min(i + batch_size, total_comentarios)
    if (i // batch_size) % log_frequency == 0 or procesados == total_comentarios:
        porcentaje = (procesados / total_comentarios) * 100
        print(f"📊 Progreso: {procesados}/{total_comentarios} comentarios ({porcentaje:.1f}%)")
        
        # Mostrar uso de memoria si es GPU
        if device == 0:
            mem = torch.cuda.memory_reserved(device) / 1e9
            print(f"💻 Uso memoria GPU: {mem:.2f} GB")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


🚀 Iniciando procesamiento de 7033 comentarios (batch_size=8)...

📊 Progreso: 8/7033 comentarios (0.1%)
💻 Uso memoria GPU: 0.80 GB
📊 Progreso: 48/7033 comentarios (0.7%)
💻 Uso memoria GPU: 16.57 GB


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


📊 Progreso: 88/7033 comentarios (1.3%)
💻 Uso memoria GPU: 16.57 GB
📊 Progreso: 128/7033 comentarios (1.8%)
💻 Uso memoria GPU: 16.58 GB
📊 Progreso: 168/7033 comentarios (2.4%)
💻 Uso memoria GPU: 16.58 GB
📊 Progreso: 208/7033 comentarios (3.0%)
💻 Uso memoria GPU: 16.58 GB
📊 Progreso: 248/7033 comentarios (3.5%)
💻 Uso memoria GPU: 16.58 GB
📊 Progreso: 288/7033 comentarios (4.1%)
💻 Uso memoria GPU: 16.58 GB
📊 Progreso: 328/7033 comentarios (4.7%)
💻 Uso memoria GPU: 16.58 GB
📊 Progreso: 368/7033 comentarios (5.2%)
💻 Uso memoria GPU: 16.58 GB
📊 Progreso: 408/7033 comentarios (5.8%)
💻 Uso memoria GPU: 16.58 GB
📊 Progreso: 448/7033 comentarios (6.4%)
💻 Uso memoria GPU: 16.58 GB
📊 Progreso: 488/7033 comentarios (6.9%)
💻 Uso memoria GPU: 16.58 GB
📊 Progreso: 528/7033 comentarios (7.5%)
💻 Uso memoria GPU: 16.58 GB
📊 Progreso: 568/7033 comentarios (8.1%)
💻 Uso memoria GPU: 16.58 GB
📊 Progreso: 608/7033 comentarios (8.6%)
💻 Uso memoria GPU: 16.58 GB
📊 Progreso: 648/7033 comentarios (9.2%)
💻 Uso mem

In [5]:
# 1. Extraer SOLO la etiqueta principal de cada resultado
etiquetas_principales = [res['labels'][0] for res in resultados]

# 2. Crear DataFrames separados
df_procesados = combined_df.iloc[:len(resultados)].copy()
df_procesados['etiqueta'] = etiquetas_principales  # Columna única con la etiqueta

df_procesados

,comment_id,comment,timestamp,is_response,response_of,case_id,conversation_id,language,emotion,etiqueta
0,Ugxl034OX9ArexHWbxB4AaABAg.9oOA2RgH9qU9wHJIOeNQ8V,Right back at you! God bless you,2023-10-25 05:05:50+00:00,True,Ugxl034OX9ArexHWbxB4AaABAg,iiTgqOUsF5s,Ugxl034OX9ArexHWbxB4AaABAg,en,Positivo,Petición de opinión
1,Ugxl034OX9ArexHWbxB4AaABAg.9oOA2RgH9qU9wrkpkIoSTy,Bless you ❤ exceptionally kind soul,2023-11-08 18:07:06+00:00,True,Ugxl034OX9ArexHWbxB4AaABAg,iiTgqOUsF5s,Ugxl034OX9ArexHWbxB4AaABAg,en,Positivo,Apoyo emocional
2,Ugxl034OX9ArexHWbxB4AaABAg.9oOA2RgH9qU9y507tBskVL,"Right back at you, dear! ❤",2023-12-09 03:30:17+00:00,True,Ugxl034OX9ArexHWbxB4AaABAg,iiTgqOUsF5s,Ugxl034OX9ArexHWbxB4AaABAg,en,Positivo,Petición de opinión
3,Ugxl034OX9ArexHWbxB4AaABAg.9oOA2RgH9qU9yZJO2VVm78,Shut up,2023-12-20 21:55:44+00:00,True,Ugxl034OX9ArexHWbxB4AaABAg,iiTgqOUsF5s,Ugxl034OX9ArexHWbxB4AaABAg,en,Negativo,Inclasificable
4,Ugxl034OX9ArexHWbxB4AaABAg.9oOA2RgH9qU9zxtGXJPp6q,Благодарю ❤️‍🩹❤️‍🩹❤️‍🩹,2024-01-24 16:50:25+00:00,True,Ugxl034OX9ArexHWbxB4AaABAg,iiTgqOUsF5s,Ugxl034OX9ArexHWbxB4AaABAg,other language,Positivo,Agradecimiento
...,...,...,...,...,...,...,...,...,...,...
7028,Ugzv8VGvIpXqGFdNgmF4AaABAg.AHLgddElqekAHLijzFXLmQ,This made me smile so much 😊💜 I&#39;m so grate...,2025-04-25 17:44:34+00:00,True,Ugzv8VGvIpXqGFdNgmF4AaABAg,lvrXIXZ_bN4,Ugzv8VGvIpXqGFdNgmF4AaABAg,en,Positivo,Intercambio de información
7029,Ugzv8VGvIpXqGFdNgmF4AaABAg.AHLgddElqekAHLzRrS7BPX,Thank you 🙏🏼,2025-04-25 20:10:30+00:00,True,Ugzv8VGvIpXqGFdNgmF4AaABAg,lvrXIXZ_bN4,Ugzv8VGvIpXqGFdNgmF4AaABAg,en,Positivo,Agradecimiento
7030,Ugzv8VGvIpXqGFdNgmF4AaABAg.AHLgddElqekAHN9DLk6c5C,🙏🏻🥰💜💜,2025-04-26 07:03:53+00:00,True,Ugzv8VGvIpXqGFdNgmF4AaABAg,lvrXIXZ_bN4,Ugzv8VGvIpXqGFdNgmF4AaABAg,other language,Positivo,Agradecimiento
7031,UgyoAS2v4m2klIP9_3J4AaABAg,"Quer entender o Mundo, pense em termos de Onda...",2022-09-28 10:17:16+00:00,False,NaN,tVffcRm_lrA,UgyoAS2v4m2klIP9_3J4AaABAg,other language,Positivo,Petición de opinión


In [6]:
df_procesados.to_csv('df_restantes.csv', index=False)

In [ ]:
combined_df.to_csv("Evaluated_scientific.csv", index=False, encoding="utf-8")

In [ ]:
combined_df

In [ ]:
combined_df